In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
import sys,os
import random
from sklearn.model_selection import train_test_split
path = os.getcwd()


In [2]:
#utility function
def getSubjectNum(fileName) :
    import re
    n = re.findall(r'\d+', fileName)
    return int(n[0])

In [3]:
def randomString(stringLength=10):
    import string
    """Generate a random string of fixed length """
    letters = string.ascii_lowercase
    return ''.join(random.choice(letters) for i in range(stringLength))


In [4]:
"""
input_filename -- The name of the input file
rps -- The number of Readings Per Second (that you want to appear in a single image)
figsize -- Figure size (in inches)
activities -- a list of numbers that represents the activities you want to consider
 
1- the function considers all the activities by default
2- it accepts multiple input files

"""
# rps : readingsPerSample
def exportFigs(input_filenames=["mHealth_subject3.log"],rps=50,figsize=(5,5), activities = range(1,13)):
    counter = 0
    
    for input_filename in input_filenames:
        subjectNum=getSubjectNum(input_filename)
        os.chdir(path)
        counter +=1
        df = pd.read_csv(input_filename, header=None, delim_whitespace=True)
        df = df[df[23]!=0] #drop 0 labeled features
        #Concatenate both lead1, lead2 readings
        features_df = pd.DataFrame(np.concatenate((df[3], df[4]), axis=0))
        #Double the features to account for lead2 readings
        targets_df = pd.DataFrame(np.concatenate((df[23], df[23]), axis=0))
        #normalization
        features_np = features_df.values
        features_np_normalized = (features_np-min(features_np))/(max(features_np)-min(features_np))
        features_df_normalized = pd.DataFrame(features_np_normalized)
        
#         segmentation
        targets_dummy  = np.array([])
        features_dummy  = np.array([])
        for i in range(int(features_df_normalized.shape[0]/rps)):
            if  float(targets_df[i*rps:i*rps+rps].mean()) in activities:
                features_dummy=np.append(features_dummy,features_df_normalized[i*rps:i*rps+rps])
                targets_dummy=np.append(targets_dummy,targets_df[i*rps:i*rps+rps].mean())
        
        features_dummy=features_dummy.reshape(-1,rps)


        #folders hiararchy
        os.chdir(path)
        outputFolderName = "test"
        try :
            os.makedirs(outputFolderName)
        except:
            pass
        path_str = os.path.join(str(path),outputFolderName)
        os.chdir(path_str)

        glob_img_counter={1:0,2:0,3:0,4:0,5:0,6:0,7:0,8:0,9:0,10:0,11:0,12:0}
        imageCounter=0
        for i in range(0,len(targets_dummy)):
            glob_img_counter[int(targets_dummy[i])]+=1
            #imageCounter= glob_img_counter[int(targets_dummy[i])]
            plt.figure(figsize=figsize)
            plt.xticks([])
            plt.yticks([])
            plt.rc("figure", facecolor="white")
            for spine in plt.gca().spines.values():
                spine.set_visible(False)
            plt.plot(features_dummy[i], color='k')
            label=str(int(targets_dummy[i]))
            print(str(imageCounter)+".jpg")
            plt.savefig(str(imageCounter)+".jpg")
            plt.clf()
            plt.cla()
            plt.close()
            imageCounter+=1
        os.chdir(path)
    return targets_dummy

In [5]:
targets = exportFigs(rps=100,input_filenames=["mHealth_subject3.log"],figsize=(3,3))

0.jpg
1.jpg
2.jpg
3.jpg
4.jpg
5.jpg
6.jpg
7.jpg
8.jpg
9.jpg
10.jpg
11.jpg
12.jpg
13.jpg
14.jpg
15.jpg
16.jpg
17.jpg
18.jpg
19.jpg
20.jpg
21.jpg
22.jpg
23.jpg
24.jpg
25.jpg
26.jpg
27.jpg
28.jpg
29.jpg
30.jpg
31.jpg
32.jpg
33.jpg
34.jpg
35.jpg
36.jpg
37.jpg
38.jpg
39.jpg
40.jpg
41.jpg
42.jpg
43.jpg
44.jpg
45.jpg
46.jpg
47.jpg
48.jpg
49.jpg
50.jpg
51.jpg
52.jpg
53.jpg
54.jpg
55.jpg
56.jpg
57.jpg
58.jpg
59.jpg
60.jpg
61.jpg
62.jpg
63.jpg
64.jpg
65.jpg
66.jpg
67.jpg
68.jpg
69.jpg
70.jpg
71.jpg
72.jpg
73.jpg
74.jpg
75.jpg
76.jpg
77.jpg
78.jpg
79.jpg
80.jpg
81.jpg
82.jpg
83.jpg
84.jpg
85.jpg
86.jpg
87.jpg
88.jpg
89.jpg
90.jpg
91.jpg
92.jpg
93.jpg
94.jpg
95.jpg
96.jpg
97.jpg
98.jpg
99.jpg
100.jpg
101.jpg
102.jpg
103.jpg
104.jpg
105.jpg
106.jpg
107.jpg
108.jpg
109.jpg
110.jpg
111.jpg
112.jpg
113.jpg
114.jpg
115.jpg
116.jpg
117.jpg
118.jpg
119.jpg
120.jpg
121.jpg
122.jpg
123.jpg
124.jpg
125.jpg
126.jpg
127.jpg
128.jpg
129.jpg
130.jpg
131.jpg
132.jpg
133.jpg
134.jpg
135.jpg
136.jpg
137.jpg
138.jp

In [6]:
import torch.nn as nn
import torch.nn.functional as F
#35
# define the CNN architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # convolutional layer (sees 72x32x4 image tensor)
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)

        # convolutional layer (sees 36x16x16 tensor)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        
        # convolutional layer (sees 18x8x32 tensor)
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        # max pooling layer
        self.pool = nn.MaxPool2d(2, 2)
        self.pool1 = nn.MaxPool2d(3, 3)
        # linear layer (64 * 4 * 4 -> 500)
        self.fc1 = nn.Linear(64 * 9 * 9, 1200)
        # linear layer (500 -> 10)
        self.fc2 = nn.Linear(1200, 700)
        self.fc3 = nn.Linear(700, 11)
        # dropout layer (p=0.25)
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        # add sequence of convolutional and max pooling layers
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
     #   x = self.pool(F.relu(self.conv4(x)))
        # flatten image input
        x = x.view(-1, 64 * 9 * 9)
        # add dropout layer
        #x = self.dropout(x)
        # add 1st hidden layer, with relu activation function
        x = F.relu(self.fc1(x))
        # add dropout layer
        x = self.dropout(x)
        # add 2nd hidden layer, with relu activation function
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
     #  x = F.relu(self.fc3(x))
     #   x = self.dropout(x)
        x = self.fc3(x)
        return x
# create a complete CNN
model = Net()
print(model)


Net(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (pool1): MaxPool2d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=5184, out_features=1200, bias=True)
  (fc2): Linear(in_features=1200, out_features=700, bias=True)
  (fc3): Linear(in_features=700, out_features=11, bias=True)
  (dropout): Dropout(p=0.2)
)


In [7]:
import torch.optim as optim

# specify loss function (categorical cross-entropy)
criterion = nn.CrossEntropyLoss()

# specify optimizer
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [8]:
from PIL import Image
import matplotlib.cm as cm
import torch

data = np.array([])
for i in range(0,len(targets)) :
    path_str = os.path.join(str(path),"test\\"+str(i)+".jpg")
    img = Image.open(path_str)
    
   # img = img.resize((16,16)).convert("L")
    image = np.asarray(img)
    print(image.shape)
 #   print("\n")
 #   image = image[:,:,0]
    data = np.append(data,image)
    
  #  data[i] = image.copy()
  #  plt.figimage(data,cmap=cm.Greys_r)
  #  plt.imshow(data[i])
  #  plt.show()
#    df = pd.read_csv(path_str, header=None, delim_whitespace=True)
    
pixels = image.shape[0]
data=data.reshape(-1,3,pixels,pixels) 
#print(data)
#print(data.shape)    


train_data, test_data, train_targets, test_targets = train_test_split(data, targets, test_size=0.2, random_state=1)

train_data, valid_data, train_targets, valid_targets = train_test_split(train_data, train_targets, test_size=0.2, random_state=1)

train_data=torch.from_numpy(train_data).float()
train_targets=torch.from_numpy(train_targets).long()

valid_data=torch.from_numpy(valid_data).float()
valid_targets=torch.from_numpy(valid_targets).long()

test_data=torch.from_numpy(test_data).float()
test_targets=torch.from_numpy(test_targets).long()


# number of epochs to train the model
n_epochs = 3000

valid_loss_min = np.Inf # track change in validation loss

for epoch in range(1, n_epochs+1):

    # keep track of training and validation loss
    train_loss = 0.0
    valid_loss = 0.0
    
    ###################
    # train the model #
    ###################
    model.train()
    # clear the gradients of all optimized variables
    optimizer.zero_grad()
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model(train_data)
    # calculate the batch loss
    loss = criterion(output, train_targets)
    # backward pass: compute gradient of the loss with respect to model parameters
    loss.backward()
    # perform a single optimization step (parameter update)
    optimizer.step()
    # update training loss
    train_loss = loss.item()
        
    ######################    
    # validate the model #
    ######################
    model.eval()
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model(valid_data)
    # calculate the batch loss
    loss = criterion(output, valid_targets)
    # update average validation loss 
    valid_loss = loss.item()
    
        
    # print training/validation statistics 
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch, train_loss, valid_loss))
    
    # save model if validation loss has decreased
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss))
        torch.save(model.state_dict(), 'model_augmented.pt')
        valid_loss_min = valid_loss

(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 

(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 216, 3)
(216, 

RuntimeError: $ Torch: not enough memory: you tried to allocate 1GB. Buy new RAM! at ..\aten\src\TH\THGeneral.cpp:201

In [ ]:
#Load the Model with the Lowest Validation Loss
model.load_state_dict(torch.load('model_augmented.pt'))

In [ ]:
# track test loss
test_loss = 0.0
class_correct = list(0. for i in range(11))
class_total = list(0. for i in range(11))

model.eval()

# forward pass: compute predicted outputs by passing inputs to the model
output = model(test_data)
# calculate the batch loss
loss = criterion(output, test_targets)
# update test loss 
test_loss = loss.item()
# convert output probabilities to predicted class
_, pred = torch.max(output, 1)    
# compare predictions to true label
correct_tensor = pred.eq(test_targets.data.view_as(pred))
correct = np.squeeze(correct_tensor.numpy()) 
# calculate test accuracy for each object class
for i in range(len(test_targets)):
    label = test_targets.data[i]
    class_correct[label] += correct[i].item()
    class_total[label] += 1

# average test loss
print('Test Loss: {:.6f}\n'.format(test_loss))

for i in range(11):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            str(i), 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (str(i)))

print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))